#Random Forest

##Import Package

In [ ]:
pip install xlrd==1.2

     |████████████████████████████████| 103 kB 5.2 MB/s 
  Attempting uninstall: xlrd
    Found existing installation: xlrd 1.1.0
    Uninstalling xlrd-1.1.0:
      Successfully uninstalled xlrd-1.1.0


In [ ]:
import pandas as pd
import numpy as np
import operator
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import classification_report
from numpy.lib.function_base import average

## Data Processing

In [ ]:
df = pd.read_excel('default of credit card clients.xls', header = 1)
df.drop(columns = 'ID', inplace = True)

In [ ]:
df = df[np.invert((df["EDUCATION"] == 0) | (df["MARRIAGE"] == 0)
                  |(df["EDUCATION"] == 5)|(df["EDUCATION"] == 6))]

df.loc[df['MARRIAGE'] == 1,'MARRIAGE'] = 'married'
df.loc[df['MARRIAGE'] == 2,'MARRIAGE'] = 'single'
df.loc[df['MARRIAGE'] == 3,'MARRIAGE'] = 'others'
df.reset_index(drop=True, inplace=True)
df = pd.get_dummies(df)
df_row = df.shape[0]
df_col = df.shape[1]
df

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,LIMIT_BAL,SEX,EDUCATION,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,...,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month,MARRIAGE_married,MARRIAGE_others,MARRIAGE_single
0,20000,2,2,24,2,2,-1,-1,-2,-2,...,0,689,0,0,0,0,1,1,0,0
1,120000,2,2,26,-1,2,0,0,0,2,...,0,1000,1000,1000,0,2000,1,0,0,1
2,90000,2,2,34,0,0,0,0,0,0,...,1518,1500,1000,1000,1000,5000,0,0,0,1
3,50000,2,2,37,0,0,0,0,0,0,...,2000,2019,1200,1100,1069,1000,0,1,0,0
4,50000,1,2,57,-1,0,-1,0,0,0,...,2000,36681,10000,9000,689,679,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29596,220000,1,3,39,0,0,0,0,0,0,...,8500,20000,5003,3047,5000,1000,0,1,0,0
29597,150000,1,3,43,-1,-1,-1,-1,0,0,...,1837,3526,8998,129,0,0,0,0,0,1
29598,30000,1,2,37,4,3,2,-1,0,0,...,0,0,22000,4200,2000,3100,1,0,0,1
29599,80000,1,3,41,1,-1,0,0,0,-1,...,85900,3409,1178,1926,52964,1804,1,1,0,0


In [ ]:
# df = df[np.invert((df["EDUCATION"] == 0) | (df["MARRIAGE"] == 0)
#                   |(df["EDUCATION"] == 5)|(df["EDUCATION"] == 6))]

# df.loc[df['SEX'] == 1,'SEX'] = 'male'
# df.loc[df['SEX'] == 2,'SEX'] = 'female'
# df.loc[df['EDUCATION'] == 1,'EDUCATION'] = 'graduate'
# df.loc[df['EDUCATION'] == 2,'EDUCATION'] = 'university'
# df.loc[df['EDUCATION'] == 3,'EDUCATION'] = 'high'
# df.loc[df['EDUCATION'] == 4,'EDUCATION'] = 'others'
# df.loc[df['MARRIAGE'] == 1,'MARRIAGE'] = 'married'
# df.loc[df['MARRIAGE'] == 2,'MARRIAGE'] = 'single'
# df.loc[df['MARRIAGE'] == 3,'MARRIAGE'] = 'others'
# df.reset_index(drop=True, inplace=True)
# df = pd.get_dummies(df)
# df_row = df.shape[0]
# df_col = df.shape[1]
# df

##Data Splitting

In [ ]:
X = df.iloc[:, df.columns != 'default payment next month']
y = df.iloc[:, df.columns == 'default payment next month']
X_train, X_rem, y_train, y_rem = train_test_split(X, y, test_size=0.2, random_state = 0)
X_valid, X_test, y_valid, y_test = train_test_split(X_rem, y_rem, test_size = 0.5, random_state = 0)
X_tra_val = np.concatenate((X_train, X_valid), axis = 0)
y_tra_val = np.concatenate((y_train, y_valid), axis = 0)

X_train = X_train.to_numpy()
X_rem = X_rem.to_numpy()
X_valid = X_test.to_numpy()
X_test = X_test.to_numpy()

y_train = y_train.to_numpy()
y_rem = y_rem.to_numpy()
y_valid = y_test.to_numpy()
y_test = y_test.to_numpy()
y_tra_val = y_tra_val

##Building Model

In [ ]:
def loss(y1, y2):
  if type(y2[0]) == np.ndarray:
      y2 = y2[0]

  y_diff = y1 - y2

  soft_accu = 1 - (y1 - y2[0]).mean()
  print("soft accu:", soft_accu)

  round_y1 = np.where(y1 >= 0.5, 1, 0)
  hard_accu = (round_y1 == y2).mean()
  print("hard accu:", hard_accu)

  return np.linalg.norm(y_diff) / np.sqrt(y_diff.shape[0])

In [ ]:
def randonForest(X_tr, X_va, X_tv, X_te, y_tr, y_va, y_tv, y_te):
  bestloss = float('inf')
  bestalpha = None
  bestn_loss = float('inf')
  bestn = None
  for alpha in range(1,10):
    regr = RandomForestRegressor(max_depth=alpha, random_state=0, n_estimators=100)
    regr.fit(X_tr, y_tr)
    y_va_pred = regr.predict(X_va)
    ls = loss(y_va_pred, y_va.T)

    print(alpha, ls)
    if ls < bestloss:
      bestloss = ls
      bestalpha = alpha

  for n_est in range(1,200):
    regr_n = RandomForestRegressor(max_depth=bestalpha, random_state=0, n_estimators=n_est)
    regr_n.fit(X_tr, y_tr)
    y_va_pred_n = regr_n.predict(X_va)
    ls_n = loss(y_va_pred_n, y_va.T)

    print(n_est, ls_n)
    if ls_n < bestn_loss:
      bestn_loss = ls_n
      bestn = n_est



  print('bestalpha', bestalpha)
  print('bestn', bestn)
  
  regr_best = RandomForestRegressor(max_depth=bestalpha, random_state=0, n_estimators=bestn)
  regr_best.fit(X_tv, y_tv)
  y_te_pred = regr_best.predict(X_te)
  y_te_pred = np.where(y_te_pred >= 0.5, 1, 0)
  print(classification_report(y_te, y_te_pred.T))
  return average(y_te_pred == y_te.T)


In [ ]:
randonForest(X_train, X_valid, X_tra_val, X_test, y_train, y_valid, y_tra_val, y_test)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


soft accu: 0.7775562133061173
hard accu: 0.8237082066869301
1 0.3791241743949204


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


soft accu: 0.7782969446793339
hard accu: 0.8250591016548463
2 0.36985967126344144


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


soft accu: 0.7772469309827839
hard accu: 0.8267477203647416
199 0.3613906185605581
bestalpha 8
bestn 61


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:34: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


              precision    recall  f1-score   support

           0       0.84      0.96      0.90      2316
           1       0.71      0.37      0.48       645

    accuracy                           0.83      2961
   macro avg       0.78      0.66      0.69      2961
weighted avg       0.81      0.83      0.81      2961



0.8291117865585951